In [21]:
pip install pyvis

Note: you may need to restart the kernel to use updated packages.


In [22]:
pip install ipysigma

Note: you may need to restart the kernel to use updated packages.


In [23]:
pip install pelote


Note: you may need to restart the kernel to use updated packages.


In [24]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import json
from pyvis.network import Network
from ipysigma import Sigma
import openpyxl


In [25]:
file_name = './Data/Bytellldp.xlsx'
df = pd.read_excel(file_name,engine='openpyxl')

In [26]:
df


,Nom Lien IP,Site Geo A,Equipement A,Classe Equipement A,Adresse IP A,Nom Interface A,Site Geo B,Equipement B,Classe Equipement B,Adresse IP B,Remarks
0,IP299645,SI028745,CSG004526,CSG,10.9.73.198,CTR03-MTL-93_LAG50,SI029340,CTR03-MTL-93,CTR,10.9.73.199,Available in lldp
1,IP299646,SI028745,CSG004526,CSG,10.9.73.200,CTR04-MTL-93_LAG50,SI029340,CTR04-MTL-93,CTR,10.9.73.201,Missing in database
2,IP191072,SI675539,CSG004129,CSG,10.101.52.104,CSG002713_LAG06:1602,SI030057,CSG002713,CSG,10.101.52.105,Available in lldp
3,IP327530,SI665166,CSG003159,CSG,10.101.47.189,CTR07-VJF-94_LAG63,SI029342,CTR07-VJF-94,CTR,10.101.47.188,Available in lldp
4,IP327531,SI665166,CSG003159,CSG,10.101.47.191,CTR08-VJF-94_LAG63,SI029342,CTR08-VJF-94,CTR,10.101.47.190,Available in lldp
...,...,...,...,...,...,...,...,...,...,...,...
43736,IP3249,SI797312,CSG003397,CSG,NaN,NaN,SI722187,CSG003358,CSG,NaN,Missing in lldp
43737,IP3250,SI774351,CSG004105,CSG,NaN,NaN,SI784617,CSG003975,CSG,NaN,Missing in lldp
43738,IP3251,SI007949,CSG004250,CSG,NaN,NaN,SI032690,CSG003323,CSG,NaN,Missing in lldp
43739,IP3252,SI031008,CSG003534,CSG,NaN,NaN,SI231279,CSG003285,CSG,NaN,Missing in lldp


In [27]:
df.columns

Index(['Nom Lien IP', 'Site Geo A', 'Equipement A', 'Classe Equipement A',
       'Adresse IP A', 'Nom Interface A', 'Site Geo B', 'Equipement B',
       'Classe Equipement B', 'Adresse IP B', 'Remarks'],
      dtype='object')

In [28]:
# df_melted=pd.melt(df,id_vars=['NODE', 'Power State', 'Status', 'Instance Name', 'Operator',
#        'Data center', 'VMS', 'VCPUS', 'RAM', 'DISK'],value_vars=['VNF','COMPUTE'])

In [29]:
# df_melted
df = df.astype(str)
# for idx, row in df.iterrows():
#     row['Src']=row['Src'].replace(' ','')
#     row['TGt']=row['TGt'].replace(' ','')
    
df=df.drop_duplicates()
    

In [30]:
df.columns


Index(['Nom Lien IP', 'Site Geo A', 'Equipement A', 'Classe Equipement A',
       'Adresse IP A', 'Nom Interface A', 'Site Geo B', 'Equipement B',
       'Classe Equipement B', 'Adresse IP B', 'Remarks'],
      dtype='object')

In [31]:
g=nx.MultiDiGraph()
from pelote import edges_table_to_graph
from pelote import tables_to_graph
from pelote import table_to_bipartite_graph

# g = edges_table_to_graph(
#   df,
#   edge_source_col="COMPUTE",
#   edge_target_col="VMS",
#   # edge_label="NODE",
#   edge_data=['NODE'],
#   directed=True
# )

table_nodes = []

table_edges = []


# for idx, row in df.iterrows():
#   node_data = {
#     'Node':row['Src'],
#     'ip':row['Srcip']
#   }
#   table_nodes.append(node_data)
  
#   node_data = {
#     'Node':row['Tgt'],
#     'ip':row['Tgtip']
#   }
#   if row['Tgtip']!='Missing':
#     table_nodes.append(node_data)
  
#   edge_data =  {
#     "source":row['Src'],
#     "target": row['Tgt'],
#     'Source_ip':row['Srcip'], 
#     'Target_ip':row['Tgtip'], 
#   }
#   if row['Tgtip']!='Missing':
#     table_edges.append(edge_data)

  
# g = tables_to_graph(
#     table_nodes, table_edges, node_col="Node", node_data=['Node', 'ip'], edge_data=['source', 'target', 'Source_ip','Target_ip'], directed=True,
# )

lookup={
    'Site Geo A' : 'source',
    'Site Geo B' : 'target'
}

col_lookup ={
    'Available in lldp':"green",
    'Missing in database':"blue",
    'Missing in lldp':"red"
    
}

edge_labels = list(df.columns)

for idx,row in df.iterrows():
    node_data={
        "Node":row['Site Geo A']
    }
    table_nodes.append(node_data)

    node_data={
        "Node":row['Site Geo B']
    }
    table_nodes.append(node_data)

    edge_data={}
    for i in edge_labels:
        if i in lookup:
            edge_data[lookup[i]]=row[i]
        else:
            edge_data[i]=row[i]
    edge_data['color']=col_lookup[row['Remarks']]
    # edge_data['color'] = row['Remarks']
    # edge_data = {
    #     "source":row['Equipement A'],
    #     "target":row['Equipement B'],
    #     # 'Nom Lien IP':row['Nom Lien IP'],
    #     # 'Etat':row['Etat'],
    #     # 'Adresse IP A':row['Adresse IP A'],
    #     # 'Constructeur A':row['Constructeur A'],
    #     # 'Classe Equipement A':row['Classe Equipement A'],
    #     # 'Classe Equipement B':['Classe Equipement B'],
    #     # 'Constructeur B':row['Constructeur B'],
    #     # 'Adresse IP B':row['Adresse IP B']
    # }
    table_edges.append(edge_data)
    

g = tables_to_graph(
    table_nodes, table_edges, node_col="Node", node_data=["Node"], edge_data=['source','target','Nom Lien IP', 'Equipement A', 'Classe Equipement A',
       'Adresse IP A', 'Nom Interface A', 'Equipement B',
       'Classe Equipement B', 'Adresse IP B', 'Remarks','color'], directed=True,
)

# g = edges_table_to_graph(
#   df,
#   edge_source_col="COMPUTE",
#   edge_target_col="VMS",
#   # edge_label="NODE",
#   edge_data=['NODE'],
#   directed=True
# )






In [32]:
# for idx, row in df.iterrows():
#     if not pd.isnull(row['primary-source-address']):  # Check if 'primary-source-address' is not NaN
#         G.add_node(row['primary-source-address'])
#         G.add_node(row['primary-destination-address'])
#         attributes = {col: row[col] for col in attribute_columns}
#         attribute_html = format_attributes(attributes)
#         G.add_edge(row['primary-source-address'], row['primary-destination-address'], title=r"{}".format(attribute_html), color='lightblue')
#     else:
#         G.add_node(row['primary-destination-address'])
#         attributes = {col: row[col] for col in attribute_columns}
#         attribute_html = format_attributes(attributes)


col_lookup ={
    'Available in lldp':"green",
    'Missing in database':"blue",
    'Missing in lldp':"red"
    
}

G=nx.MultiDiGraph()
def format_attributes(data_dict):
    formatted_string = ""
    for key, value in data_dict.items():
        if key=='value':
            formatted_string += f"{'source'}: {value}\n"
        else:
            formatted_string += f"{key}: {value}\n"
    return formatted_string

attribute_columns = ['Site Geo A','Site Geo B','Nom Lien IP', 'Equipement A', 'Classe Equipement A','Adresse IP A', 'Nom Interface A', 'Equipement B','Classe Equipement B', 'Adresse IP B', 'Remarks']
# for idx, row in df.iterrows():
#     dest = row['TGt']
#     src=row['Src']
#     attributes = {col: row[col] for col in attribute_columns}
#     attribute_html = format_attributes(attributes)
#     G.add_node(dest, color='Green')
#     if not pd.isnull(src):
#         G.add_node(src,shape = 'triangle',color='Orange')
#         G.add_edge(src,dest,title=r"{}".format(attribute_html))
    


for idx, row in df.iterrows():

    src_name = row['Equipement A']
    dest_name = row['Equipement B']
    G.add_node(src_name, type =row['Classe Equipement A'] )
    G.add_node(dest_name,type = row['Classe Equipement B'])
    attributes = {col: row[col] for col in attribute_columns}
    attribute_html = format_attributes(attributes)

    G.add_edge(src_name,dest_name, title=r"{}".format(attribute_html),Site_Geo_A = ['Site Geo A'],Site_Geo_B = ['Site Geo B'],Nom_Lien_IP=row['Nom Lien IP'],Equipment_A = row['Equipement A'],Classe_Equipement_A=row['Classe Equipement A'],Adresse_IP_A=row['Adresse IP A'],Nom_Interface_A=row['Nom Interface A'],Equipement_B=row['Equipement B'],Classe_Equipement_B=row['Classe Equipement B'],Adresse_IP_B=row['Adresse IP B'],Remarks = row['Remarks'],color=col_lookup[row['Remarks']])
import os
G = nx.relabel_nodes(G, {n: str(n) for n in G.nodes()})
nt = Network(height="1500px", width="100%", directed=True, notebook=True, filter_menu=True, cdn_resources='remote')
nt.from_nx(G)
nt.barnes_hut(gravity=-18950,central_gravity=2.0,spring_length=200,spring_strength=0.5,damping=0.75,overlap=0.5)
nt.set_edge_smooth('dynamic')
# Show additional buttons for physics and interaction (optional)
nt.show_buttons(filter_=['physics', 'interaction'])
# Generate and save the HTML file
html_file_path = 'Bytellldp.html'
# if os.path.exists(html_file_path):
#     # Delete the file if it exists
#     os.remove(html_file_path)
# Write the HTML file
# nx.write_gml(G,html_file_path)
nt.write_html(html_file_path)

In [33]:
# sigma = Sigma(g,clickable_edges=True)

# Save the Sigma object to an HTML file
print(g._node)
Sigma.write_html(g,"./templates/files/Bytellldp.html",fullscreen=True,clickable_edges=True,node_size=g.degree,node_color='red',raw_edge_color='color')

{'SI028745': {'Node': 'SI028745'}, 'SI029340': {'Node': 'SI029340'}, 'SI675539': {'Node': 'SI675539'}, 'SI030057': {'Node': 'SI030057'}, 'SI665166': {'Node': 'SI665166'}, 'SI029342': {'Node': 'SI029342'}, 'SI863317': {'Node': 'SI863317'}, 'SI145846': {'Node': 'SI145846'}, 'SI032568': {'Node': 'SI032568'}, 'SI151070': {'Node': 'SI151070'}, 'SI813325': {'Node': 'SI813325'}, 'SI029346': {'Node': 'SI029346'}, 'SI874061': {'Node': 'SI874061'}, 'SI034456': {'Node': 'SI034456'}, 'SI130244': {'Node': 'SI130244'}, 'SI130796': {'Node': 'SI130796'}, 'SI030634': {'Node': 'SI030634'}, 'SI882219': {'Node': 'SI882219'}, 'SI029345': {'Node': 'SI029345'}, 'SI042888': {'Node': 'SI042888'}, 'SI124794': {'Node': 'SI124794'}, 'SI033229': {'Node': 'SI033229'}, 'SI030848': {'Node': 'SI030848'}, 'SI036387': {'Node': 'SI036387'}, 'SI035512': {'Node': 'SI035512'}, 'SI859337': {'Node': 'SI859337'}, 'SI033299': {'Node': 'SI033299'}, 'SI035582': {'Node': 'SI035582'}, 'SI029391': {'Node': 'SI029391'}, 'SI001232': {